# Face Recognition - Generate Eigenface Classifiers

This notebook tests face detection and recognition from sample images

In [1]:
import cv2, cv2.face
import os, os.path
import numpy as np
from PIL import Image

In [73]:
# constants
CASCADE_PATH = './data/haarcascades/'
CASCADE_NAME = './haarcascade_frontalface_default.xml'
SAMPLE_TRAINING = './data/sample_training/'
SAMPLE_TESTING = './data/sample_testing/'

In [3]:
# Viola-Jones classifier for Haar feature extraction 
cascader = cv2.CascadeClassifier(os.path.join(CASCADE_PATH, CASCADE_NAME))
cascader_kwargs = { # setup detection args here
    'scaleFactor': 1.1,
    'minNeighbors': 1,
    'flags': 0
}

In [4]:
# face recognizer
recognizer = cv2.face.createEigenFaceRecognizer()

In [58]:
# function that gets images with respective labels from a given folder
def get_images_and_labels(folder, debug=False, debug_faces=True):
    
    labels = []
    images = []
    
    for label in os.listdir(folder):
        
        if debug: print(label)
        added = 0
        path = os.path.join(folder, label)
        
        if os.path.isdir(path):
            
            for filename in os.listdir(path):
                
                ok = 0

                try:
                    image_path = os.path.join(path, filename)

                    # get grayscale image
                    image_pil = Image.open(image_path).convert('L')
                    image = np.array(image_pil, 'uint8')
                    
                    # Alternative for Eigenfaces
                    images.append(image)
                    labels.append(label)
                    ok = 1
                    
                    '''

                    # get the face using Viola-Jones detector
                    faces = cascader.detectMultiScale(image, **cascader_kwargs)
                    for (x, y, w, h) in faces:
                        images.append(image[y:y+h, x:x+w])
                        labels.append(label)
                        if debug_faces:
                            cv2.imshow('Adding faces to training set...', image[y:y+h, x:x+w])
                            cv2.waitKey(50)
                    ok += len(faces)
                    '''
                    

                except:
                    pass
                
                if debug: print('\t' + filename + (' [%d face]' % ok))
                if ok: added += 1
        
        if debug: print('[Added %d images]' % added)
    
    if debug_faces:
        cv2.destroyAllWindows()
        
    return images, labels
    

In [48]:
%%time
# try getting labels from sample images folder
training_set = get_images_and_labels(SAMPLE_TRAINING, debug=False, debug_faces=False)
print('Collected %d images\n' % len(training_set[0]))

Collected 75 images

CPU times: user 140 ms, sys: 8 ms, total: 148 ms
Wall time: 175 ms


In [49]:
%%time
# Perform the training
images, labels = training_set
recognizer.train(images, np.array(range(len(images))))
print('Trained %d images\n' % len(images))

Trained 75 images

CPU times: user 472 ms, sys: 88 ms, total: 560 ms
Wall time: 555 ms


In [59]:
# setup label dictionary and testing set
testing_set = get_images_and_labels(SAMPLE_TESTING, debug=False, debug_faces=False)
label_dict = {i: label for i, label in enumerate(labels)}

In [65]:
help(recognizer.predict)

Help on built-in function predict:

predict(...) method of cv2.face_BasicFaceRecognizer instance
    predict(src) -> label, confidence



In [74]:
%%time
# Perform testing

images, labels = training_set
print('Performing %d tests\n' % len(images))

correct = 0
total = 0


for image, expected_label in zip(images, labels):
    label, confidence = recognizer.predict(image)
    actual_label = label_dict[label]
    if actual_label == expected_label:
        correct += 1
        print('%s is correctly recognized with confidence %.10f' % (actual_label, confidence))
    else:
        print('%s incorrect (recognized as %s with confidence %.10f)' % (actual_label, expected_label, confidence))
    total += 1

print()
print('Accuracy: %.2f%%' % (100 * (correct / max(1, total))))
print('Correct:  %d' % correct)
print('Wrong:    %d' % (total - correct))

Performing 75 tests

Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Hosni_Mubarak is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized with confidence 0.0000000000
Jeong_Se-hyun is correctly recognized wit